In [18]:
pip show sklearn

Name: sklearn
Version: 0.0.post1
Summary: deprecated sklearn package, use scikit-learn instead
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.11/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [19]:
import numpy as np
import pandas as pd
import chess
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
%matplotlib inline

In [20]:
values = {
    'P': 1, 'p': -1,
    'N': 3, 'n': -3,
    'B': 3, 'b': -3,
    'R': 5, 'r': -5,
    'Q': 9, 'q': -9,
}

In [21]:
df = pd.read_csv('data/chess.csv')
df.head()

,white_username,black_username,white_id,black_id,white_rating,black_rating,white_result,black_result,time_class,time_control,rules,rated,fen,pgn
0,-Amos-,miniman2804,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/miniman2804,1708,1608,win,checkmated,daily,1/259200,chess,True,r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R ...,"[Event ""Enjoyable games 2 - Round 1""]\n[Site ""..."
1,-Amos-,koltcho69,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/koltcho69,1726,1577,win,resigned,daily,1/172800,chess,True,8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33,"[Event ""Rapid Rats - Board 5""]\n[Site ""Chess.c..."
2,-Amos-,enhmandah,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/enhmandah,1727,842,win,resigned,daily,1/172800,chess,True,rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/...,"[Event ""CHESS BOARD CLASH - Round 1""]\n[Site ""..."
3,enhmandah,-Amos-,https://api.chess.com/pub/player/enhmandah,https://api.chess.com/pub/player/-amos-,819,1727,checkmated,win,daily,1/172800,chess,True,r3kb1r/pp3ppp/3p1n2/2pKp3/P3P3/1P6/4qP1P/QNB5 ...,"[Event ""CHESS BOARD CLASH - Round 1""]\n[Site ""..."
4,-Amos-,Shalllow-Blue,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/shalllow-blue,1729,1116,win,resigned,daily,1/172800,chess,True,r3b2r/pp6/2pPpR1k/4n3/2P3Q1/3B4/PP4PP/R5K1 b -...,"[Event ""CHESS BOARD CLASH - Round 1""]\n[Site ""..."


In [22]:
def fen_to_piece_count(fen):
    board = chess.Board(fen)
    pieces = {
        'P': len(board.pieces(chess.PAWN, chess.WHITE)),
        'p': len(board.pieces(chess.PAWN, chess.BLACK)),
        'N': len(board.pieces(chess.KNIGHT, chess.WHITE)),
        'n': len(board.pieces(chess.KNIGHT, chess.BLACK)),
        'B': len(board.pieces(chess.BISHOP, chess.WHITE)),
        'b': len(board.pieces(chess.BISHOP, chess.BLACK)),
        'R': len(board.pieces(chess.ROOK, chess.WHITE)),
        'r': len(board.pieces(chess.ROOK, chess.BLACK)),
        'Q': len(board.pieces(chess.QUEEN, chess.WHITE)),
        'q': len(board.pieces(chess.QUEEN, chess.BLACK)),
    }
    return pieces

def get_result(result):
    if result == 'win':
        return 1
    elif result == 'draw':
        return 0.5
    else:
        return 0

In [23]:
X = df['fen'].apply(fen_to_piece_count).apply(pd.Series)
y = df['result'] = df['white_result'].apply(get_result)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7651764354066986


In [25]:
piece_values = model.coef_[0]
piece_values_normalized = {
    'P': 1,
    'p': -1,
    'N': piece_values[2] / piece_values[0],
    'n': piece_values[3] / piece_values[1],
    'B': piece_values[4] / piece_values[0],
    'b': piece_values[5] / piece_values[1],
    'R': piece_values[6] / piece_values[0],
    'r': piece_values[7] / piece_values[1],
    'Q': piece_values[8] / piece_values[0],
    'q': piece_values[9] / piece_values[1],
}

print("Piece values in pawn units:")
for piece, value in piece_values_normalized.items():
    print(f"{piece}: {value:.2f}")

Piece values in pawn units:
P: 1.00
p: -1.00
N: 1.89
n: 2.65
B: 2.06
b: 2.71
R: 3.23
r: 3.83
Q: 5.94
q: 7.73


In [26]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.19121162783710322


In [27]:
# Calculate the average piece count
average_piece_count = np.mean(X_train, axis=0)

# Calculate the expected game outcome for a reference position
reference_position = np.array(average_piece_count).reshape(1, -1)
reference_outcome = model.predict(reference_position)

# Find the effect of adding or removing a piece
piece_values = {}
for i, piece in enumerate(['P', 'p', 'N', 'n', 'B', 'b', 'R', 'r', 'Q', 'q']):
    test_position = reference_position.copy()
    test_position[0, i] += 1 if piece.isupper() else -1
    test_outcome = model.predict(test_position)
    piece_values[piece] = test_outcome - reference_outcome

# Normalize piece values based on pawn values
piece_values_normalized = {
    piece: value / piece_values['P']
    for piece, value in piece_values.items()
}

print("Piece values in pawn units:")
for piece, value in piece_values_normalized.items():
    print(f"{piece}: {value[0]:.2f}")

Piece values in pawn units:
P: 1.00
p: 2.30
N: 3.17
n: -5.21
B: -5.10
b: 3.37
R: -3.28
r: 1.43
Q: -0.51
q: -5.21


/usr/local/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does